# Gesture Volume Control

In [1]:
import cv2
import numpy as np
import time
import HandTrackingModule as htm
import math

In [3]:
# Webcam Weight and Height
wCam, hCam = 648, 488



cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)

pTime = 0

detector = htm.handDetector(detectionCon=0.7)

# PyCaw : https://github.com/AndreMiras/pycaw
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate( IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
# volume.GetMute()
# volume.GetMasterVolumeLevel()
volRange = volume.GetVolumeRange()

minVol = volRange[0]
maxVol = volRange[1]
vol = 0
volBar = 400
volPer = 0

while True:
    success, img = cap.read()

    img = detector.findHands(img)
    lmList = detector.findPosition(img, draw=False)
    if len(lmList) != 0:
        #print(lmList[4], lmList[8])

        # [4] = Ibu jari, [8] = Telunjuk
        x1, y1 = lmList[4][1], lmList[4][2]
        x2, y2 = lmList[8][1], lmList[8][2]
        cx, cy = (x1+x2) // 2, (y1+y2) // 2

        # Menggambar bulatan pada ibu jari dan telunjuk
        cv2.circle(img, (x1,y1), 15, (255,0,255), cv2.FILLED)
        cv2.circle(img, (x2,y2), 15, (255,0,255), cv2.FILLED)
        # menambahkan garis yang menghubungkan antara 2 jari
        cv2.line(img, (x1,y1), (x2,y2), (255,0,255), 3)
        # menambahkan titik tengah pada garis
        cv2.circle(img, (cx, cy), 15, (255,0,255), cv2.FILLED)

        length = math.hypot(x2-x1, y2-y1)
        #print(length)

        # Hand range 10 - 200 : Hand range bisa berbeda-beda,
        # pakai print(length) untuk mengetahui hand range
        # Volume Range -65 - 0

        vol = np.interp(length, [10, 200], [minVol, maxVol])
        volBar = np.interp(length, [10, 200], [400, 150])
        volPer = np.interp(length, [10, 200], [0, 100])
        print(int(length), vol)
        volume.SetMasterVolumeLevel(vol, None)

        if length<50:
            cv2.circle(img, (cx, cy), 15, (0, 255, 0), cv2.FILLED)

    cv2.rectangle(img, (50, 150), (85, 400), (255,0,0), 3)
    cv2.rectangle(img, (50, int(volBar)), (85, 400), (255,0,0), cv2.FILLED)
    cv2.putText(img, f'{int(volPer)} %', (40,450), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 3)



    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    cv2.putText(img, f'FPS: {int(fps)}', (48,50), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 3)
    

    cv2.imshow("Image", img)
    cv2.waitKey(1)

170 -10.223899836923472
164 -12.363157894736837
172 -9.334931784960276
172 -9.570882278588549
175 -8.31532170774993
171 -9.770731825381574
174 -8.749611215080392
172 -9.43039345722211
173 -8.995102409590814
170 -10.063735681911979
173 -9.090019719588774
175 -8.453209567886638
174 -8.749611215080392
175 -8.409168532743763
173 -9.090019719588774
165 -11.945897392373233
149 -17.220299856271552
111 -30.55828641962796
42 -54.07367271633244
22 -60.83796007645863
27 -59.310639998177464
29 -58.543108371787056
28 -58.850128164433876
79 -41.51268128365001
161 -13.121061465204626
186 -4.7331612901445155
196 -1.330770258882346
201 0.0
203 0.0
205 0.0
205 0.0
204 0.0
206 0.0
205 0.0
206 0.0
206 0.0
206 0.0
206 0.0
206 0.0
206 0.0
205 0.0
207 0.0
197 -0.8344305053609133
125 -25.603001035987127
82 -40.39415220922341
20 -61.60440415601178
23 -60.64492979816362
23 -60.60103894936765
20 -61.51336842414302
75 -42.72693977651636
155 -15.328911707455624
185 -5.002982548636588
197 -0.8344305053609133
200 0.

KeyboardInterrupt: 